<!-- $$V_+(k)=\sqrt{2\pi}\sigma Ae^{-\pi(\sqrt{2\pi}\sigma)^2k^2}$$

$$V_+(x)=Ae^{-\frac{x^2}{2\sigma^2}}$$

$$V_+(k_x,k_y)=(\sqrt{2\pi}\sigma)^2 Ae^{-\pi(\sqrt{2\pi}\sigma)^2({k_x}^2+{k_y}^2)}$$

$$V_+(x,y)=Ae^{-\frac{x^2+y^2}{2\sigma^2}}$$ -->

$$V_+(x,y)=Ae^{-\frac{x^2+y^2}{2\sigma^2}}=Ae^{-\pi\frac{x^2+y^2}{(\sqrt{2\pi}\sigma)^2}}$$
$$V_+(k_x,k_y)=A(\sqrt{2\pi}\sigma)^2e^{-\pi(\sqrt{2\pi}\sigma)^2({k_x}^2+{k_y}^2)}$$

In [3]:
import numpy as np
import tensorflow as tf

import plotly.graph_objects as go

In [4]:
num_quasimomenta = 49
max_momentum_site = 9

max_momentum_site_change = 2 * max_momentum_site
num_momentum_sites = (max_momentum_site_change + 1) ** 2

In [5]:
quasimomentum = np.mgrid[0:2:num_quasimomenta*1j,
                         0:2:num_quasimomenta*1j].reshape(2,-1).T

momentum_site = np.mgrid[-max_momentum_site:max_momentum_site+1:1,
                         -max_momentum_site:max_momentum_site+1:1].reshape(2,-1).T

momentum = 2*momentum_site + quasimomentum[:, np.newaxis]
kinetic = np.linalg.norm(momentum, axis=-1)**2

momentum_site_change = momentum_site[:, np.newaxis] - momentum_site

In [6]:
def render_momentum_site_potential(attractive_strength, repulsive_strength, repulsive_spread):
    attractive_potential = -1/16 * attractive_strength * np.pad(np.array([
        [1, 0, 2, 0, 1],
        [0, 0, 0, 0, 0],
        [2, 0, 4, 0, 2],
        [0, 0, 0, 0, 0],
        [1, 0, 2, 0, 1]
    ]), max_momentum_site_change - 2)

    momentum_site = np.meshgrid(*(np.arange(-max_momentum_site_change, max_momentum_site_change+1),)*2)
    repulsive_potential = repulsive_strength * (np.sqrt(2*np.pi) * repulsive_spread)**2 * np.exp(-np.pi * (np.sqrt(2*np.pi) * repulsive_spread)**2 * (momentum_site[0]**2 + momentum_site[1]**2))

    return attractive_potential + repulsive_potential

In [7]:
def compute_dispersion(momentum_site_potential):
    potential = momentum_site_potential[
        momentum_site_change[:, :, 0] + max_momentum_site_change,
        momentum_site_change[:, :, 1] + max_momentum_site_change,
    ]

    energy = tf.linalg.diag(kinetic) + potential[np.newaxis, ...]

    dispersion = tf.reshape(tf.linalg.eigvalsh(energy), (num_quasimomenta, num_quasimomenta, num_momentum_sites))

    return dispersion

In [91]:
def compute_tunneling(dispersion):
    tunneling = np.abs(np.fft.fftshift(np.fft.fft2(np.fft.ifftshift(dispersion)))) / num_quasimomenta**2
    tunneling[num_quasimomenta//2, num_quasimomenta//2] = np.nan
    return tunneling

# Analysis

The width of the flat band decreases as the depth of the square lattice increases for sufficiently strong Gaussian depth. 

In [141]:
momentum_site_potential = render_momentum_site_potential(60, 60, 0.08)

spatial_potential = np.real(np.fft.fftshift(np.fft.fft2(np.fft.ifftshift(momentum_site_potential))))

dispersion = compute_dispersion(momentum_site_potential)

flat_band = dispersion[:, :, 1]
width = np.max(flat_band) - np.min(flat_band)
tunneling = compute_tunneling(flat_band)

lateral_tunneling = np.abs(tunneling[num_quasimomenta//2-1, num_quasimomenta//2])
diagonal_tunneling = np.abs(tunneling[num_quasimomenta//2-1, num_quasimomenta//2-1])

print(f"Width: {width:.4f}")

Width: 0.0047


In [142]:
figure = go.Figure(data=[go.Surface(z=spatial_potential)])
figure.update_layout(title="Potential", 
                     autosize=False, width=750, height=750)
figure.show()

figure = go.Figure(data=[go.Surface(z=dispersion[:, :, band], showscale=False) for band in range(3)])
figure.update_layout(title="Dispersion",
                     autosize=False, width=750, height=750)
figure.show()

radius = 3
figure = go.Figure(data=[go.Surface(z=tunneling[num_quasimomenta//2-radius:num_quasimomenta//2+radius+1, num_quasimomenta//2-radius:num_quasimomenta//2+radius+1])])
figure.update_layout(title="Tunneling",
                     autosize=False, width=750, height=750)
figure.show()

In [151]:
spatial_potential[:, spatial_potential.shape[1]//2].shape

(37,)

In [152]:
figure = go.Figure(data=[go.Scatter(y=spatial_potential[:, spatial_potential.shape[1]//2])])
figure.update_layout(title="Potential", 
                     autosize=False, width=750, height=750)
figure.show()

figure = go.Figure(data=[go.Scatter(y=dispersion[:, num_quasimomenta//2, band]) for band in range(3)])
figure.update_layout(title="Dispersion",
                     autosize=False, width=750, height=750)
figure.show()